## Kitty treniranje


Ova bilježnica samo istreniran kitty mrežu.

In [1]:
!pip install torchsummary

In [2]:
import subprocess
import shutil
import torch
import torchvision
from torchvision import transforms
from torchsummary import summary
import json
import pickle

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
!nvidia-smi

Thu Apr 14 12:18:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:84:00.0 Off |                    0 |
| N/A   49C    P0    35W / 250W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
model_name = "kitty"

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = torchvision.datasets.ImageFolder(root='dataset/train', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='dataset/test', transform=transform)


In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=64, shuffle=True, num_workers=4)

In [8]:
print(len(train_loader))

5836


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.AvgPool2d(4, 4)
        
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.AvgPool2d(4, 4)
        
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.AvgPool2d(4, 4)
        
        self.conv3_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.AvgPool2d(2 , 2)
        
        self.fc = nn.Linear(324, 4)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv1_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv2_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv3_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x


model = Net()
model = model.to(device)

In [10]:
summary(model, (3, 640, 640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [-1, 3, 640, 640]               6
            Conv2d-2          [-1, 9, 638, 638]             252
         AvgPool2d-3          [-1, 9, 159, 159]               0
       BatchNorm2d-4          [-1, 9, 159, 159]              18
            Conv2d-5         [-1, 16, 157, 157]           1,312
         AvgPool2d-6           [-1, 16, 39, 39]               0
       BatchNorm2d-7           [-1, 16, 39, 39]              32
            Conv2d-8           [-1, 25, 37, 37]           3,625
         AvgPool2d-9             [-1, 25, 9, 9]               0
      BatchNorm2d-10             [-1, 25, 9, 9]              50
           Conv2d-11             [-1, 36, 7, 7]           8,136
        AvgPool2d-12             [-1, 36, 3, 3]               0
           Linear-13                    [-1, 4]           1,300
Total params: 14,731
Trainable params: 

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.2)

In [12]:
early_batches_acc = {1, 2, 4, 6, 8, 10, 15, 20, 30, 50,
                                75, 100, 150, 200, 300, 400, 500,
                               600, 700, 800}

torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_0_batch_0.pth')

In [13]:
test_accs = dict()

for epoch in range(10):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if (i % 1000 == 1) or (epoch == 0 and (i in early_batches_acc)):
            correct = 0
            total = 0
            with torch.no_grad():
                for ix, data in enumerate(test_loader):
                    if ix > 100:
                        break
                    images, labels = data
                    images, labels = images.cuda(), labels.cuda()
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            acc = correct / total
            test_accs[(epoch, i)] = acc
            print(f'Accuracy of the network on the test images: {100*acc} %')
            with open(f'accuracies_{model_name}.pickle', 'wb') as fp:
                pickle.dump(test_accs, fp)
        if epoch == 0 and (i % 2 == 1):
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')
        
        if epoch == 1 and i % 20 == 0:
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        if epoch > 1 and i % 40 == 0:
            torch.save(model.state_dict(), f'saved_models/{model_name}/epoch_{epoch}_batch_{i}.pth')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

    scheduler.step()

print('Finished Training')

Accuracy of the network on the test images: 29.176980198019802 %
Accuracy of the network on the test images: 30.987004950495052 %
Accuracy of the network on the test images: 32.53403465346535 %
Accuracy of the network on the test images: 34.42141089108911 %
Accuracy of the network on the test images: 38.211633663366335 %
Accuracy of the network on the test images: 39.99071782178218 %
Accuracy of the network on the test images: 42.09467821782179 %
Accuracy of the network on the test images: 42.40408415841584 %
Accuracy of the network on the test images: 46.93688118811881 %
Accuracy of the network on the test images: 52.6144801980198 %
Accuracy of the network on the test images: 51.74814356435643 %
Accuracy of the network on the test images: 58.09096534653465 %
Accuracy of the network on the test images: 53.650990099009896 %
[1,   200] loss: 1.018
Accuracy of the network on the test images: 57.58044554455446 %
Accuracy of the network on the test images: 64.23267326732673 %
[1,   400] los

[7,  4000] loss: 0.093
Accuracy of the network on the test images: 82.99814356435643 %
[7,  4200] loss: 0.099
[7,  4400] loss: 0.091
[7,  4600] loss: 0.089
[7,  4800] loss: 0.089
[7,  5000] loss: 0.096
Accuracy of the network on the test images: 82.82797029702971 %
[7,  5200] loss: 0.094
[7,  5400] loss: 0.092
[7,  5600] loss: 0.091
[7,  5800] loss: 0.088
Accuracy of the network on the test images: 82.99814356435643 %
[8,   200] loss: 0.089
[8,   400] loss: 0.097
[8,   600] loss: 0.087
[8,   800] loss: 0.084
[8,  1000] loss: 0.096
Accuracy of the network on the test images: 82.90532178217822 %
[8,  1200] loss: 0.092
[8,  1400] loss: 0.091
[8,  1600] loss: 0.093
[8,  1800] loss: 0.093
[8,  2000] loss: 0.094
Accuracy of the network on the test images: 82.95173267326733 %
[8,  2200] loss: 0.090
[8,  2400] loss: 0.088
[8,  2600] loss: 0.092
[8,  2800] loss: 0.095
[8,  3000] loss: 0.093
Accuracy of the network on the test images: 83.94183168316832 %
[8,  3200] loss: 0.096
[8,  3400] loss: 0

In [14]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
confusion_matrix = torch.zeros(4, 4)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
print(confusion_matrix.diag()/confusion_matrix.sum(1))

Accuracy of the network on the test images: 83 %
tensor([0.8225, 0.9237, 0.7399, 0.8651])


In [15]:
confusion_matrix

tensor([[51945.,  2507.,  4474.,  4232.],
        [ 1890., 58032.,  1939.,   965.],
        [ 3667.,  3861., 55103., 11843.],
        [ 2441.,  1935.,  4306., 55670.]])